In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import sys

In [2]:
mode = "Moisture Mode"

diri = "/scratch/gb02/mr4682/data/regridded/UM/"
diro = "/scratch/gb02/mr4682/data/regridded/UM/composites/"

filei = f"{diri}olr.zoom10.to.0p25deg.anom.nc"
var_name = "rlut_anom"

In [3]:
ds = xr.open_dataset(filei)

In [4]:
time_start = "2020-01-20 00:00:00"
time_end = "2020-03-31 21:00:00"

In [5]:
time = ds["time"].sel(time=slice(time_start, time_end))
lat = ds["latitude"]
lon = ds["longitude"]

In [6]:
field = ds[var_name].sel(time=slice(time_start, time_end))
field

<xarray.DataArray 'rlut_anom' (time: 576, latitude: 249, longitude: 1440)> Size: 2GB
[206530560 values with dtype=float64]
Coordinates:
    cell       (latitude, longitude) int64 3MB ...
    crs        float64 8B ...
  * latitude   (latitude) float64 2kB -30.0 -29.75 -29.5 ... 31.5 31.75 32.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * time       (time) datetime64[ns] 5kB 2020-01-20 ... 2020-03-31T21:00:00
Attributes:
    STASH:          [  1   2 205]
    UM_name:        toa_outgoing_longwave_flux
    grid_mapping:   healpix_nested
    healpix_zoom:   10
    long_name:      toa_outgoing_longwave_flux
    source:         Data from Met Office Unified Model
    standard_name:  toa_outgoing_longwave_flux
    um_version:     13.5
    units:          W m-2

In [7]:
if mode == "Moisture Mode":
    mode_str = "moisture.mode"
    mode_var_name = "moisture_mode"
elif mode == "Mixed System":
    mode_str = "mixed.system"
    mode_var_name = "mixed_system"
elif mode == "IG Wave":
    mode_str = "ig.wave"
    mode_var_name = "ig_wave"
elif mode == "Tropical Cyclone":
    modes_str = "tc"
    mode_var_name = mode_str
else:
    sys.exit(f"There is no mode named {mode}!")

In [8]:
track = pd.read_csv(f"{diri}tracks/Track.olr.zoom10.to.0p25deg.{mode_str}.csv")

In [9]:
max_activity = pd.read_csv(f"{diri}tracks/max.activity.olr.zoom10.to.0p25deg.{mode_str}.csv")
max_activity = max_activity.loc[(pd.to_datetime(max_activity["timestr"]) >= pd.to_datetime(time_start)) & (pd.to_datetime(max_activity["timestr"]) <= pd.to_datetime(time_end) - pd.Timedelta(60, "d"))]
max_activity

,Unnamed: 0.1,Unnamed: 0,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,longitude,cell,time_cell
0,204,248,8,33,32.668902,947.024950,198,-2.5,249,2020-01-21 00:00:00,2020-01-21 00:00:00,-21.832774,56.756238,2,1 days 00:00:00
1,35,50,2,12,74.214585,486.623478,74,-2.0,51,2020-01-20 06:00:00,2020-01-20 06:00:00,-11.446354,-58.344130,4,0 days 06:00:00
2,310,380,11,75,92.123032,817.084625,46,-3.0,381,2020-01-21 09:00:00,2020-01-21 09:00:00,-6.969242,24.271156,5,1 days 09:00:00
3,83,110,4,21,121.711410,150.546224,113,-2.0,111,2020-01-20 12:00:00,2020-01-20 12:00:00,0.427852,-142.363444,7,0 days 12:00:00
4,274,335,10,53,122.925319,645.063714,18,-2.5,336,2020-01-21 06:00:00,2020-01-21 06:00:00,0.731330,-18.734071,8,1 days 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,3344,3732,91,87,42.504534,1275.869097,16,-3.0,3733,2020-01-31 09:00:00,2020-01-31 09:00:00,-19.373866,138.967274,585,0 days 00:00:00
256,3461,3854,93,116,24.643528,148.698333,65,-3.5,3855,2020-01-31 15:00:00,2020-01-31 15:00:00,-23.839118,-142.825417,586,0 days 06:00:00
257,3425,3815,93,8,43.942373,621.572757,86,-2.0,3816,2020-01-31 15:00:00,2020-01-31 15:00:00,-19.014407,-24.606811,587,0 days 03:00:00
263,3493,3888,94,81,245.647840,120.807298,136,-2.5,3889,2020-01-31 18:00:00,2020-01-31 18:00:00,31.411960,-149.798176,598,0 days 00:00:00


In [10]:
season = "ALL"  # ["ALL", "DJF", "MAM", "JJA", "SON"]
domain = "EQ"   # ["EQ", "NH", "SH"]

x_size = 15.0
y_size = 15.0

grid_res = lon.values[1] - lon.values[0]
time_res = (time.values[1] - time.values[0]) / np.timedelta64(1, "h")

lagmax = 2

In [11]:
if season == "ALL":
    max_activity_in_season = max_activity
elif season == "DJF":
    max_activity_in_season = max_activity.loc[(pd.to_datetime(max_activity["timestr"]).dt.month == 12) | (pd.to_datetime(max_activity["timestr"]).dt.month <= 2)]
elif season == "MAM":
    max_activity_in_season = max_activity.loc[(pd.to_datetime(max_activity["timestr"]).dt.month >= 3) & (pd.to_datetime(max_activity["timestr"]).dt.month <= 5)]
elif season == "JJA":
    max_activity_in_season = max_activity.loc[(pd.to_datetime(max_activity["timestr"]).dt.month >= 6) & (pd.to_datetime(max_activity["timestr"]).dt.month <= 8)]
elif season == "SON":
    max_activity_in_season = max_activity.loc[(pd.to_datetime(max_activity["timestr"]).dt.month >= 9) & (pd.to_datetime(max_activity["timestr"]).dt.month <= 11)]

In [12]:
if domain == "EQ":
    max_activity_in_domain = max_activity_in_season.loc[(max_activity_in_season["latitude"] > -10.0) & (max_activity_in_season["latitude"] < 10.0)]
    long_name = "Instances of Equatorial " + field.attrs["long_name"] + " (" + season + ")"
elif domain == "NH":
    max_activity_in_domain = max_activity_in_season.loc[(max_activity_in_season["latitude"] >= 10.0) & (max_activity_in_season["latitude"] <= 20.0)]
    long_name = "Instances of Northern Hemisphere " + field.attrs["long_name"] + " (" + season + ")"
elif domain == "SH":
    max_activity_in_domain = max_activity_in_season.loc[(max_activity_in_season["latitude"] >= -20.0) & (max_activity_in_season["latitude"] <= -10.0)]
    long_name = "Instances of Southern Hemisphere " + field.attrs["long_name"] + " (" + season + ")"

In [13]:
n_Features = len(max_activity_in_domain["timestr"].values)

x = np.arange(-x_size, x_size + grid_res, grid_res)
y = np.arange(-y_size, y_size + grid_res, grid_res)

In [14]:
instances = np.zeros((n_Features, 2 * lagmax + 1, len(y), len(x))) * np.nan

In [15]:
lat3 = np.zeros(len(lat.values) + int(2*y_size/grid_res))
lat3[0:int(y_size/grid_res)] = np.min(lat.values) + y[0:int(y_size/grid_res)]
lat3[int(y_size/grid_res):-int(y_size/grid_res)] = lat.values
lat3[-int(y_size/grid_res):] = np.max(lat.values) + y[-int(y_size/grid_res):]

lon3 = np.zeros(len(lon.values) * 3)
lon3[0*len(lon.values):1*len(lon.values)] = lon.values - 360.0
lon3[1*len(lon.values):2*len(lon.values)] = lon.values
lon3[2*len(lon.values):3*len(lon.values)] = lon.values + 360.0

In [18]:
for i in range(0, n_Features):
    track_i = track.loc[track["cell"] == max_activity_in_domain["cell"].values[i]]
    max_activity_i = max_activity_in_domain.loc[max_activity_in_domain["cell"] == max_activity_in_domain["cell"].values[i]]
    max_activity_index = np.where(track_i["timestr"].values == max_activity_i["timestr"].values)[0][0]

    time_i = track_i["timestr"].values
    lat_i = track_i["latitude"].values
    lon_i = track_i["longitude"].values

    field_i = np.zeros((len(time_i), len(lat3), len(lon3))) * np.nan
    field_i[:, int(y_size/grid_res):-int(y_size/grid_res), 0*len(lon.values):1*len(lon.values)] = field.sel(time=slice(time_i[0], time_i[-1])).values
    field_i[:, int(y_size/grid_res):-int(y_size/grid_res), 1*len(lon.values):2*len(lon.values)] = field.sel(time=slice(time_i[0], time_i[-1])).values
    field_i[:, int(y_size/grid_res):-int(y_size/grid_res), 2*len(lon.values):3*len(lon.values)] = field.sel(time=slice(time_i[0], time_i[-1])).values
    
    for lag in range(-lagmax, lagmax + 1):
        if (max_activity_index + lag >= 0) & (max_activity_index + lag < len(time_i)):
            lat_i_lag = lat_i[max_activity_index + lag]
            lon_i_lag = lon_i[max_activity_index + lag]

            lat_i_lag_index = np.argmin(np.abs(lat3 - lat_i_lag))
            lon_i_lag_index = np.argmin(np.abs(lon3 - lon_i_lag))
            print(f"cell = {track_i['cell'].values}, lag = {lag}, (lat, lon) = ({lat_i_lag}, {lon_i_lag}), (lat_index, lon_index) = ({lat_i_lag_index}, {lon_i_lag_index})")

            instances[i, lag + lagmax, :, :] = field_i[max_activity_index + lag, lat_i_lag_index-int(y_size/grid_res):lat_i_lag_index+int(y_size/grid_res)+1, lon_i_lag_index-int(x_size/grid_res):lon_i_lag_index+int(x_size/grid_res)+1]

cell = [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5], lag = -2, (lat, lon) = (-7.036788126402342, 23.80442616721274), (lat_index, lon_index) = (152, 2255)
cell = [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5], lag = -1, (lat, lon) = (-6.997023814900395, 24.054893948098737), (lat_index, lon_index) = (152, 2256)
cell = [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5], lag = 0, (lat, lon) = (-6.969241999736099, 24.271156221365487), (lat_index, lon_index) = (152, 2257)
cell = [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5], lag = 1, (lat, lon) = (-6.964548162152191, 24.445755710112508), (lat_index, lon_index) = (152, 2258)
cell = [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5], lag = 2, (lat, lon) = (-6.965973926413177, 24.588837170097264), (lat_index, lon_index) = (152, 2258)
cell = [7 7 7 7 7 7 7 7], lag = -2, (lat, lon) = (0.4983180069838191, -142.335099804482), (lat_index, lon_index) = (182, 1591)
cell = [7 7 7 7 7 7 7 7], lag = -1, (lat, lon) = (0.4408604220056489, -142.3369927218265), (lat_index, lon_index) = (182, 1591)

ValueError: could not broadcast input array from shape (121,0) into shape (121,121)

In [19]:
track.loc[track["cell"] == 104]

,Unnamed: 0,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,latitude,longitude,cell,time_cell
311,381,11,76,103.817628,1428.181070,187,-3.0,382,2020-01-21 09:00:00,2020-01-21 09:00:00,-4.045593,177.045268,104,0 days 00:00:00
346,420,12,76,104.253044,1429.425740,217,-3.0,421,2020-01-21 12:00:00,2020-01-21 12:00:00,-3.936739,177.356435,104,0 days 03:00:00
385,462,13,75,104.776394,1430.634933,233,-3.0,463,2020-01-21 15:00:00,2020-01-21 15:00:00,-3.805902,177.658733,104,0 days 06:00:00
431,511,14,87,107.463963,1431.987574,12,-3.5,512,2020-01-21 18:00:00,2020-01-21 18:00:00,-3.134009,177.996894,104,0 days 09:00:00
466,548,15,89,107.718997,1433.318267,35,-3.5,549,2020-01-21 21:00:00,2020-01-21 21:00:00,-3.070251,178.329567,104,0 days 12:00:00
499,582,16,86,108.063768,1434.619470,54,-3.5,583,2020-01-22 00:00:00,2020-01-22 00:00:00,-2.984058,178.654868,104,0 days 15:00:00
529,618,17,87,108.536195,1435.825657,74,-3.5,619,2020-01-22 03:00:00,2020-01-22 03:00:00,-2.865951,178.956414,104,0 days 18:00:00
558,660,18,91,109.027189,1436.886837,94,-3.5,661,2020-01-22 06:00:00,2020-01-22 06:00:00,-2.743203,179.221709,104,0 days 21:00:00
588,694,19,82,109.398493,1437.800963,111,-3.5,695,2020-01-22 09:00:00,2020-01-22 09:00:00,-2.650377,179.450241,104,1 days 00:00:00
620,732,20,88,109.689617,1438.573937,122,-3.5,733,2020-01-22 12:00:00,2020-01-22 12:00:00,-2.577596,179.643484,104,1 days 03:00:00


In [25]:
cell_to_use = []

for cell in max_activity["cell"].values:
    track_i = track.loc[track["cell"] == cell]
    if ~(np.any(np.isnan(track_i["latitude"].values)) | np.any(np.isnan(track_i["longitude"].values))):
        cell_to_use.append(cell)

cell_to_use

[2,
 4,
 5,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 19,
 23,
 26,
 29,
 33,
 34,
 40,
 41,
 45,
 46,
 48,
 54,
 55,
 58,
 59,
 62,
 63,
 65,
 66,
 69,
 70,
 74,
 75,
 77,
 81,
 82,
 90,
 91,
 93,
 94,
 96,
 100,
 105,
 106,
 108,
 110,
 112,
 113,
 115,
 117,
 118,
 119,
 121,
 122,
 123,
 124,
 125,
 128,
 131,
 133,
 135,
 141,
 151,
 153,
 155,
 157,
 160,
 161,
 185,
 186,
 187,
 189,
 193,
 198,
 201,
 203,
 205,
 206,
 210,
 214,
 215,
 222,
 223,
 224,
 227,
 228,
 229,
 230,
 233,
 234,
 235,
 236,
 238,
 239,
 241,
 242,
 244,
 245,
 246,
 248,
 249,
 254,
 256,
 260,
 261,
 263,
 268,
 271,
 273,
 274,
 277,
 278,
 281,
 282,
 283,
 287,
 288,
 290,
 293,
 296,
 297,
 299,
 300,
 302,
 304,
 306,
 311,
 312,
 315,
 316,
 318,
 321,
 323,
 325,
 326,
 327,
 328,
 330,
 331,
 333,
 334,
 337,
 338,
 341,
 342,
 343,
 345,
 347,
 348,
 349,
 350,
 351,
 356,
 358,
 359,
 360,
 361,
 363,
 365,
 366,
 367,
 370,
 378,
 379,
 383,
 388,
 389,
 392,
 393,
 394,
 396,
 